# Whisker Frames - Symbolic Notebook
2025 - For generating the equations of the Whisker Frames project

In [1]:
# !pip install sympy

In [2]:
#imports
import sympy as sym
from sympy.solvers import solve
from sympy import symbols, Matrix, sin, cos, tan, cot, atan, simplify, expand, Poly, Eq, integrate

## Some helper functions for Equation display

In [3]:
def multi_Eq(sides):
    if len(sides) == 1:
        raise ValueError('must have at least 2 inputs')
    elif len(sides) == 2:
        return Eq(*sides,evaluate=False)
    elif len(sides) > 2:
        return Eq(sides[0],multi_Eq(sides[1:]),evaluate=False)
    else:
        raise ValueError('not the right input')
        
def Eqdisp(*args,label=None,linebreak=True):
    multi_list = [symbols(arg) if type(arg)==str else arg for arg in args] #convert any strings to symbols
    if len(multi_list) == 1:
        display_Eq = multi_list[0]
    else:
        display_Eq = multi_Eq(multi_list)
    #display!
    if label is not None:
        print(label)
    display(display_Eq)
    if linebreak:
        print('\n')

## Symbols and setup

In [4]:
#display variables
r_d,u_d,w_d,phi_d = symbols('\mathbf{r},\mathbf{u},\mathbf{w},\Phi')
#configuration variables
r1,r2,th,u1,u2,w1,w2,a,b,p1,p2 = symbols(r'r1,r2,\theta,u1,u2,w1,w2,\alpha,\beta,p1,p2',real=True)
#parameter and other variables
s,h,x,y = symbols(r's,h,x,y',real=True)
lam = symbols(r'\lambda')

## 2.2: A vector model for the mechanism configuration and whisker orientations

In [5]:
#r vector
r = Matrix([r1,r2])

#"x" configuration
x_conf = Matrix([r1,r2,th])

#define w
w = Matrix([w1,w2])
w_x = Matrix([s*sin(th), 1-s*cos(th)])

#define u as function of r1,r2,theta and as function of r1,r2,w1,w2
u_x = w_x*h - r
u_w = w*h - r

In [6]:
#define protraction in multiple forms
prot_x = atan((u_x[1])/(u_x[0]))
prot_w = atan((u_w[1])/(u_w[0]))
prot_u = atan((u2)/(u1))

#define spread
sprd_x = sym.diff(prot_x,h)
sprd_w = simplify(sym.diff(prot_w,h))

In [7]:
#2.2 equation displays
disps = True
if disps:
    #definitions
    Eqdisp('r',r,label='the vectors r and w',linebreak=False)
    Eqdisp('w',w_x,w)
    
    #u
    Eqdisp('u',symbols('w')*h - symbols('r'),u_w,label='the vector u is defined as a function of r and w')
    
    #phi and dphi
    Eqdisp(r'\Phi',prot_u,prot_w,label='Phi, or protraction, is defined like so')
    Eqdisp(r'\frac{\mathrm{d}\Phi}{\mathrm{d}h}',sprd_w,label='the derivative of phi, spread, is defined like so')

the vectors r and w


Eq(r, Matrix([
[r1],
[r2]]))

Eq(w, Eq(Matrix([
[     s*sin(\theta)],
[-s*cos(\theta) + 1]]), Matrix([
[w1],
[w2]])))



the vector u is defined as a function of r and w


Eq(u, Eq(h*w - r, Matrix([
[h*w1 - r1],
[h*w2 - r2]])))



Phi, or protraction, is defined like so


Eq(\Phi, Eq(atan(u2/u1), atan((h*w2 - r2)/(h*w1 - r1))))



the derivative of phi, spread, is defined like so


Eq(\frac{\mathrm{d}\Phi}{\mathrm{d}h}, (-w1*(h*w2 - r2) + w2*(h*w1 - r1))/((h*w1 - r1)**2 + (h*w2 - r2)**2))

## 2.3 Generalizing the analysis to every point in the x-y plane reveals that every control frame configuration is associated with a parabola

In [8]:
#the arbitrary point will be defined here as z = (x,y)
z = Matrix([x,y])

#q vector
q = Matrix([x,y-h])

#magnitude of cross(u,q)
ucrossq_x = (u_x[0]*q[1])-(u_x[1]*q[0])
ucrossq_w = (u_w[0]*q[1])-(u_w[1]*q[0])

#solve for h when ucrossq = 0
solveh_x = solve(ucrossq_x,h)
solveh_w = solve(ucrossq_w,h)

In [9]:
#get the discriminant (inside square root term) of the solution
sqrt_term_x = solveh_x[0].args[3].args[2].args[1]
sqrt_term_w = -solveh_w[0].args[2].args[1]

#the conic equation is equal to the descriminant
conic_x = sqrt_term_x**2
conic_w = sqrt_term_w**2

#wrap the conic in a Polynomial object
conic_poly_x = Poly(conic_x, [x,y])
conic_poly_w = Poly(conic_w, [x,y])

#Get coefficients
coeffs_x = conic_poly_x.coeffs()
coeffs_w = conic_poly_w.coeffs()

In [10]:
#piece together the matrix form 
a,b,d,c,e,f = coeffs_x
A_x = Matrix([[a,b/2,d/2],
              [b/2,c,e/2],
              [d/2,e/2,f]])
a,b,d,c,e,f = coeffs_w
A_w = Matrix([[a,b/2,d/2],
              [b/2,c,e/2],
              [d/2,e/2,f]])

#classification
A33 = A_w[0:2,0:2]
detA33 = A33.det()

#degenerate conditions
degeneracy = A_w.det().simplify()

In [11]:
#2.3 equation displays
disps = True
if disps:
    #setup
    Eqdisp('q',q,label='the vector q')
    Eqdisp(ucrossq_w,label='the value of |u x q|')
    
    #solving for the parabola
    Eqdisp('h',solveh_w[0],label='solving for h when u x q = 0')
    Eqdisp('conic',conic_w,label='value of the discriminant is a conic section (parabola)')
    
    #matrix form and degeneracy
    Eqdisp('conic',r'{z^T}Az',label='we can express the parabola in matrix form',linebreak=False)
    Eqdisp('A',A_w,linebreak=False)
    Eqdisp('z',Matrix([x,y,1]))
    Eqdisp(degeneracy,0,label='the parabola is degenerate when the following expression is satisfied')
    
    

the vector q


Eq(q, Matrix([
[     x],
[-h + y]]))



the value of |u x q|


-x*(h*w2 - r2) + (-h + y)*(h*w1 - r1)



solving for h when u x q = 0


...

Eq(conic, {z^T}Az)

Eq(A, Matrix([
[           w2**2, -w1*w2, -r1*w2 + 2*r2*w1],
[          -w1*w2,  w1**2,           -r1*w1],
[-r1*w2 + 2*r2*w1, -r1*w1,            r1**2]]))

Eq(z, Matrix([
[x],
[y],
[1]]))



the parabola is degenerate when the following expression is satisfied


Eq(4*w1**2*(-r1**2*w2**2 + 2*r1*r2*w1*w2 - r2**2*w1**2), 0)

## 2.4. A coordinate transformation relates each control frame configuration to the position and orientation of its associated parabola.

In [12]:
#find the axes of the parabola via principal axis theorem (only doing this for A_w)
A33 = A_w[0:2,0:2]
A_ev = A33.eigenvects()
evec1,evec2 = A_ev[0][2][0], A_ev[1][2][0]

#define change-of-basis matrix W
W = Matrix([[w2,w1],[-w1,w2]])
W_inv = W.inv()

#define p vector
p_p = Matrix([p1,p2])

#get p to r conversions
r_p = W*p_p
p_r = W_inv*r
p_x = simplify(p_r.subs({w1:w_x[0],w2:w_x[1]}))

#new configuration
x_3_3 = Matrix([p1,p2,th])

In [13]:
#2.4 displays
Eqdisp(r'A_{33}',A33,label='The 2x2 submatrix')
Eqdisp(r'v_{1}',evec1,label='Calculate the eigenvectors of A33: v1 and v2',linebreak=False)
Eqdisp(r'v_{2}',evec2)
Eqdisp('W',W,label='The change of basis matrix')
Eqdisp('r','Wp',r_p,label='r in terms of p')
Eqdisp('p',r'W^{-1}r',p_r,'parabolicfocus',label='p in terms of r')
Eqdisp('p',p_x,label='p in terms of r, s, theta')

The 2x2 submatrix


Eq(A_{33}, Matrix([
[ w2**2, -w1*w2],
[-w1*w2,  w1**2]]))



Calculate the eigenvectors of A33: v1 and v2


Eq(v_{1}, Matrix([
[w1/w2],
[    1]]))

Eq(v_{2}, Matrix([
[-w2/w1],
[     1]]))

...

Eq(r, Eq(Wp, Matrix([
[ p1*w2 + p2*w1],
[-p1*w1 + p2*w2]])))



p in terms of r


Eq(p, Eq(W^{-1}r, Eq(Matrix([
[r1*w2/(w1**2 + w2**2) - r2*w1/(w1**2 + w2**2)],
[r1*w1/(w1**2 + w2**2) + r2*w2/(w1**2 + w2**2)]]), parabolicfocus)))



p in terms of r, s, theta


Eq(p, Matrix([
[(-r1*s*cos(\theta) + r1 - r2*s*sin(\theta))/(s**2 - 2*s*cos(\theta) + 1)],
[ (r1*s*sin(\theta) - r2*s*cos(\theta) + r2)/(s**2 - 2*s*cos(\theta) + 1)]]))

## 2.5 The new coordinate system allows for independent control of protraction and fanning

In [14]:
#get u in terms of p. Simplify using U matrix
u_p = w*h-r_p
U = Matrix([[h-p2,-p1],[p1,h-p2]])
# simplify(u_p[0]**2 + u_p[1]**2)

#calculate protraction and spread
prot_p = atan((u_p[1])/(u_p[0]))
sprd_p = simplify(sym.diff(prot_p,h))

#more justification for sprd_p
sprd_num = expand(u_p[0]**2)
sprd_denom = expand(u_p[0]**2 + u_p[1]**2)
# Eqdisp('num',sprd_num)
# Eqdisp('denom',sprd_denom)

#differentiating phi with respect to theta...
dphi_dth = simplify(prot_p.subs({w1:w_x[0],w2:w_x[1]}).diff(th))

#verifying the expression in (31) is a correct antiderivative (this expression was obtained by hand, verified here)
soln_31 = th-sym.atan((s-cos(th))/(sin(th)))
is_d31_30 = dphi_dth.equals(soln_31.diff(th))

In [15]:
#2.5 displays
Eqdisp('u',symbols('w')*h-symbols('Wp'),u_p,'Uw',label='the u vector in terms of p',linebreak=False)
Eqdisp('U',U)
Eqdisp(symbols('u')-symbols('Uw'),simplify(u_p-U*w),label='check that matrix U is correctly defined')
Eqdisp('Phi',prot_p,label='protraction with respect to p')
Eqdisp(r'\frac{\mathrm{d}\Phi}{\mathrm{d}h}',sprd_p,label='spread with respect to p')
Eqdisp('(31)',soln_31,label="equation 31 is the antiderivative of dphi/dth. (hard-coded here, obtained on paper)")
Eqdisp(is_d31_30,label="is the expression in (31) a correct antiderivative of dphi/dth?")

the u vector in terms of p


Eq(u, Eq(-Wp + h*w, Eq(Matrix([
[h*w1 - p1*w2 - p2*w1],
[h*w2 + p1*w1 - p2*w2]]), Uw)))

Eq(U, Matrix([
[h - p2,    -p1],
[    p1, h - p2]]))



check that matrix U is correctly defined


Eq(-Uw + u, Matrix([
[0],
[0]]))

...

Eq(\frac{\mathrm{d}\Phi}{\mathrm{d}h}, -p1/(h**2 - 2*h*p2 + p1**2 + p2**2))



equation 31 is the antiderivative of dphi/dth. (hard-coded here, obtained on paper)


Eq((31), \theta - atan((s - cos(\theta))/sin(\theta)))



is the expression in (31) a correct antiderivative of dphi/dth?


True

## 4.1.  Proof that all whisker lines will be tangent to the parabola 

The envelope conditions are...

$\left\{\begin{matrix}
G(x,y,t) = 0
\\ 
\frac{\partial F}{\partial t} = 0
\end{matrix}\right.$

Where G is a family of curves, here specifically its the whisker lines. h is the single parameter that defines the family of lines. 

In [16]:
#define G(x,y,h)
G = (u_w[1]/u_w[0])*x - y + h

#now multiply it by u2 since you can do that in the equation equal to zero...
G_mult = sym.collect(sym.expand(sym.simplify(G*u_w[0])),h)

#take the derivative of G_mult with respect to h to form the second envelope condition
condition2 = G_mult.diff(h)

#solve for h
h_solve = solve(condition2,h)[0]

#solve system of eqns by substituting h into G=0
conditions_solve = sym.simplify(G_mult.subs({h:h_solve}))

#again, since its equal to zero, we can multiply ("both sides") by (-4w1)
conditions_solve_mult = conditions_solve*(-4*w1)

# is it equal to the conic?
is_equal_to_conic = conditions_solve_mult.equals(conic_w)

In [17]:
#4.1 displays
Eqdisp("G",(u2/u1)*x-y+h,G,label='The family of curves G(x,y,h)')
Eqdisp(r'\frac{\mathrm{d}G}{\mathrm{d}h}',0,condition2,label='Derivative of G with respect to h (second envelope condition)')
Eqdisp('h_G',h_solve, label='first condition (G=0) solved for h')
Eqdisp(conditions_solve_mult,0,label = 'solution to system of equations')
Eqdisp(is_equal_to_conic,label='is it equal to the conic?')

The family of curves G(x,y,h)


Eq(G, Eq(h - y + u2*x/u1, h + x*(h*w2 - r2)/(h*w1 - r1) - y))



Derivative of G with respect to h (second envelope condition)


Eq(\frac{\mathrm{d}G}{\mathrm{d}h}, Eq(0, 2*h*w1 - r1 - w1*y + w2*x))



first condition (G=0) solved for h


Eq(h_G, (r1 + w1*y - w2*x)/(2*w1))



solution to system of equations


Eq(-4*w1*(r1*y - r2*x - (r1 + w1*y - w2*x)**2/(4*w1)), 0)



is it equal to the conic?


True

## 4.2.  Proof that the principal axes of the parabola are spanned by w and ⊥w

In [18]:
# take the eigenvalues of the A33 () matrix...
v1 = A33.eigenvects()[0][2][0]
v2 = A33.eigenvects()[1][2][0]

#multiply by chosen constants 
basis_1 = v1*w2
basis_2 = v2*w1

In [19]:
#4.2 displays
Eqdisp('v_1',v1,label='eigenvalues of A33')
Eqdisp('v_2',v2)
Eqdisp(r'\mathrm{basis_1}',basis_1,label='chosen basis vectors')
Eqdisp(r'\mathrm{basis_2}',basis_2)

eigenvalues of A33


Eq(v_1, Matrix([
[w1/w2],
[    1]]))

Eq(v_2, Matrix([
[-w2/w1],
[     1]]))



chosen basis vectors


Eq(\mathrm{basis_1}, Matrix([
[w1],
[w2]]))

Eq(\mathrm{basis_2}, Matrix([
[-w2],
[ w1]]))

## 4.3. Proof that the focus of the parabola is located at the coordinates (p1,p2) in the x-y plane.

In [20]:
#invert A
A_w_inv = A_w.inv()

#get values...
K = A_w_inv[1,1]/2
H = A_w_inv[0,1]
S = A_w_inv[1,2]
R = A_w_inv[0,2]

#calculate focus...
focus = sym.simplify((-K + H*sym.I)/(R + S*sym.I))

#get real and imaginary parts
xF = sym.re(focus)
yF = sym.im(focus)

In [21]:
#4.3 displays
Eqdisp('A^-1',A_w_inv,label='Inverse of A')
Eqdisp('F',focus,label='focus of the parabola')
Eqdisp('x_F','p_1',xF,label='The real and imaginary parts of this focus are equal to the coordinates of the point (p1,p2)')
Eqdisp('x_F','p_2',yF)

Inverse of A


Eq(A^-1, Matrix([
[                            0, -r1/(2*r1*w1*w2 - 2*r2*w1**2),        -1/(2*r1*w2 - 2*r2*w1)],
[-r1/(2*r1*w1*w2 - 2*r2*w1**2),     -r2/(r1*w1*w2 - r2*w1**2), -w2/(2*r1*w1*w2 - 2*r2*w1**2)],
[       -1/(2*r1*w2 - 2*r2*w1), -w2/(2*r1*w1*w2 - 2*r2*w1**2),                             0]]))



focus of the parabola


Eq(F, (I*r1 - r2)/(w1 + I*w2))



The real and imaginary parts of this focus are equal to the coordinates of the point (p1,p2)


Eq(x_F, Eq(p_1, r1*w2/(w1**2 + w2**2) - r2*w1/(w1**2 + w2**2)))

Eq(x_F, Eq(p_2, r1*w1/(w1**2 + w2**2) + r2*w2/(w1**2 + w2**2)))

## 4.4. Proof that as θ changes with p_1and p_2  held constant, the control frame rotates about (p_1, p_2), the focus of the parabola

In [22]:
#substitute s and theta terms into the p-space expression for r
r_p_subs = r_p.subs({w1:w_x[0],w2:w_x[1]})

#verify that the O matrix, derived from the P matrix (see paper), is orthogonal
P = Matrix([[-p1,p2],[-p2,-p1]])
O = sym.simplify(P/(sym.sqrt(p1**2 + p2**2)))
is_equal_to_I = (O.T*O).equals(sym.eye(2))

In [23]:
Eqdisp('r',r_p_subs,label='r, in p space, expressed in s and theta terms')
Eqdisp('O',O,label='O matrix')
Eqdisp(is_equal_to_I,label='is O orthogonal?')

r, in p space, expressed in s and theta terms


Eq(r, Matrix([
[ p1*(-s*cos(\theta) + 1) + p2*s*sin(\theta)],
[-p1*s*sin(\theta) + p2*(-s*cos(\theta) + 1)]]))



O matrix


Eq(O, Matrix([
[-p1/sqrt(p1**2 + p2**2),  p2/sqrt(p1**2 + p2**2)],
[-p2/sqrt(p1**2 + p2**2), -p1/sqrt(p1**2 + p2**2)]]))



is O orthogonal?


True